# Classificação de imagens com CIFAR-10

Inicia-se conectando com seu Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Instalando joblib para armazenar e recuperar objetos

In [2]:
!pip install joblib

Importações da primeira etapa(construção do modelo):

In [39]:
import tensorflow as tf # biblioteca para deep learning
from tensorflow.keras import datasets, layers, models # importação de datasets, camadas pré-definidas, definição e treino de modelos
import keras # api para construção e treinamento do modelo
# import psutil
import joblib # importação para salvar e carregar modelos
# import time
import numpy as np # biblioteca para operações com algebra linear

In [40]:
# Carrega o dataset CIFAR-10 que tem 60 mil imagens de 32x32 pixels
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data() # o load_data() retorna duas tuplas

# ao treinar contendo imagens e labels, os valores dos pixels das imagens de treino e teste são normalizados
train_images, test_images = train_images / 255.0, test_images / 255.0

In [41]:
model = models.Sequential() # Cria um modelo sequencial, que é uma pilha linear de camadas

# Adiciona uma camada convolucional com 64 filtros de tamanho 5x5 com função de ativação ReLU e entrada para imagens 32x32 com 3 canais(RGB)
model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(32, 32, 3)))

model.add(layers.MaxPooling2D((2, 2))) # Adiciona uma camada de pooling que reduz as dimensões espaciais sobre 2x2 pixels

model.add(layers.Conv2D(128, (5, 5), activation='relu')) # Adiciona outra camada convolucional com 128 filtros de tamanho 5x5

model.add(layers.MaxPooling2D((2, 2))) # Adiciona outra camada de pooling que reduz as dimensões pela metade

model.add(layers.Conv2D(128, (5, 5), activation='relu')) # Adiciona mais uma camada convolucional com 128 filtros de tamanho 5x5

model.add(layers.Flatten()) # Achata a saída da última camada convolucional transformando a matriz em 1D

model.add(layers.Dense(128, activation='relu')) # Adiciona uma camada densa (totalmente conectada) com 128 unidades

# Adiciona uma camada densa com 10 unidades e função de ativação softmax para calcular a distribuição nas 10 classes
model.add(layers.Dense(10, activation='softmax'))


In [42]:
# Compila o modelo usando o otimizador adam para ajustar a taxa de aprendizado
# É usado 'SparseCategoricalCrossentropy' como função de perda para classificação
# Já 'from_logits=True' indica que a função de perda espera saídas como logits(funções lineares) ao invés de probabilidades
# Como padrão, usa-se accuracy como métrica pra medir a precisão do modelo durante o treino
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [43]:
# Aqui se inicia o treinamento do modelo com images e labels que são os dados usados
# foi setado que o treino passe 5 vezes pelo conjunto de dados e um tamanho de amostra de 64 antes de passar pelo backpropagation
history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


782/782 [==============================] - 233s 296ms/step - loss: 1.5480 - accuracy: 0.4332 - val_loss: 1.3074 - val_accuracy: 0.5290
Epoch 2/5
782/782 [==============================] - 252s 323ms/step - loss: 1.2031 - accuracy: 0.5735 - val_loss: 1.1568 - val_accuracy: 0.5976
Epoch 3/5
782/782 [==============================] - 265s 339ms/step - loss: 1.0432 - accuracy: 0.6332 - val_loss: 1.0243 - val_accuracy: 0.6386
Epoch 4/5
782/782 [==============================] - 244s 312ms/step - loss: 0.9256 - accuracy: 0.6748 - val_loss: 1.0374 - val_accuracy: 0.6408
Epoch 5/5
782/782 [==============================] - 253s 324ms/step - loss: 0.8357 - accuracy: 0.7053 - val_loss: 0.9252 - val_accuracy: 0.6779


In [44]:
# Aqui é definido uma parada que interrompe caso a loss seja muito ruim
# restore_best_weights=True faz com que o modelo retorne aos melhores pesos
# start_from_epoch=2 inicia a verificação de parada antecipada a partir da segunda época
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, start_from_epoch=2)

In [45]:
model.save('model.h5') # Salva o modelo treinado no arquivo model.h5, armazenando pesos, arquitetura e otimizador do modelo

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Implementação de API usando Flask e Ngrok

## Instalação para rodar o ngrok

Instalação para o Flask criar tunel público para a aplicação, normalmente usado para ambientes de desenvolvimento que não possuem endereço IP público como é o caso desse modelo desenvolvido neste notebook Colab

In [10]:
!pip install flask-ngrok

Instalação do ngrok

In [11]:
!pip install pyngrok

In [12]:
# improtação do ngrok para criar e gerenciar túneis que apontam para serviços para rodar localmente aplicações
# os tuneis são encapsulamento de dados privados para que sejam enviados para uma rede pública
from pyngrok import ngrok

Atualização do ngrok caso necessário

In [13]:
!pip install --upgrade pyngrok

### Importações necessárias para rodar o Flask

In [14]:
from flask import Flask, request, jsonify # microframework para criar aplicações web
# importação de threads para o flask rodar nessas threads e permitir que células posteriores possam rodar e para que o servidor responda outras requisições
from threading import Thread
from flask_ngrok import run_with_ngrok # importação para integrar a aplicação flask com o ngrok e iniciar os tuneis
from PIL import Image # importação para manipulação de imagens
import io # manipula streams de dados binários para imagens em memória

Caso for necessário e ultrapassar o limite grátis de 3 tuneis rodando disponibilizados pelo ngrok, é necessário desconectar todos os tuneis

### Atenção: Rode essa célula apenas se for necessário, como o caso acima

In [15]:
tunnels = ngrok.get_tunnels() # recupera uma lista de todos os túneis ativos

# Desconecta cada túnel individualmente
for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

Conexão com os tuneis do ngrok, com a porta padrão 5000

In [29]:
http_tunnel = ngrok.connect(5000)  # Conecta um novo túnel à porta 5000 local

In [30]:
print("Túnel reaberto:", http_tunnel.public_url)  # Imprime o URL público do túnel para ser usada futuramente

Túnel reaberto: https://1509-34-105-96-145.ngrok-free.app


É importante destacar que antes de fazer a conexão com o ngrok, é preciso efetuar o cadastro no site Ngrok e ter uma conta que terá o token de acesso, este token é disponibilizado no dashboard na opção "Your Authtoken", que é onde se tem o token abaixo:

In [31]:
ngrok.set_auth_token('2hbscXmCDNgwgyJaAW3tyVOES4C_39ii3MLoEiv5GURCX83RJ') # token de acesso do ngrok para ser possível se conectar ao servidor

In [32]:
tunnels = ngrok.get_tunnels() # obtém uma lista de todos os túneis ngrok ativos

Inicialização da aplicação Flask genérica, o qual carrega o modelo pré-treinado e define uma rota chamada "predict" que aceita apenas requsiições POST, visto que é a única que vai ser trabalhada(envio de imagem)

In [26]:
app = Flask(__name__)

# Carrega o modelo
model = tf.keras.models.load_model('/content/model.h5')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'erro': 'Nenhum arquivo'}), 400

    # Obtém o arquivo da requisição
    file = request.files['file']
    if file:
        image = Image.open(io.BytesIO(file.read())) # abre imagem
        image = image.resize((32, 32)) # redimensiona a imagem no padrão definido como 32x32
        image = np.array(image)
        image = image / 255.0  # Normalização
        image = np.expand_dims(image, axis=0)  # Adiciona dimensão de batch

        predictions = model.predict(image) # faz a previsão do modelo
        predicted_class = np.argmax(predictions, axis=1)[0]  # Classificação

        return jsonify({'predicted_class': str(predicted_class)})

# execução
def run_flask():
    app.run(threaded=True, use_reloader=False, port=5000)

# Roda Flask em uma thread separada para permitir a rodagem das próximas células
flask_thread = Thread(target=run_flask)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


Instalação para que seja possível requisições como POST serem feitas

In [27]:
!pip install requests


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [22]:
import requests # importação para requisições, auxiliando o input de imagens para classificação

In [38]:
url = 'https://1509-34-105-96-145.ngrok-free.app/predict'

# Caminho para a imagem a ser enviada
image_path = '/content/drive/MyDrive/CIFAR-10/thumb-horse.jpg'

# Abre a imagem em modo binário
with open(image_path, 'rb') as img:
    files = {'file': ('image.jpg', img, 'image/jpeg')}

    # Envia a requisição POST
    response = requests.post(url, files=files)

    # Imprime a resposta da API
    print(response.text)

1/1 [==============================] - 0s 30ms/step


INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 16:27:36] "POST /predict HTTP/1.1" 200 -


{"predicted_class":"7"}

